In [197]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

In [198]:
def add_oracle(circuit, start):
    circuit.cx(start, start + 2)
    circuit.ccx(start, start + 2, start + 3)
    circuit.cx(start + 1, start + 2)
    circuit.ccx(start + 1, start + 2, start + 3)

In [199]:
from qiskit.circuit.library import C4XGate, C3XGate

In [200]:
def find_the_largest_number(a, b):
    a_num = a
    b_num = b
    
    # Get the bit lengths of a and b
    a_bin = bin(abs(a)).replace("0b", "")
    b_bin = bin(abs(b)).replace("0b", "")
    
    a_bits = len(a_bin) + 1
    b_bits = len(b_bin) + 1
    
    n_bits = (max(a_bits, b_bits) * 3 + 1) + 1
    
    # Create a quantum circuit with n qubits and 1 classical bit
    qc = QuantumCircuit(n_bits, 1)
    
    if a < 0:
        a_neg = True
    else:
        a_neg = False
    
    if b < 0:
        b_neg = True
    else:
        b_neg = False
    
    #add_oracle(qc, 0)
    
    a = str(bin(abs(a)))[2:]
    b = str(bin(abs(b)))[2:]
    
    if len(a) < len(b):
        while(len(a) < len(b)):
            a = "0" + a
    elif len(b) < len(a):
        while(len(b) < len(a)):
            b = "0" + b
    
    if a_neg:
        a = "1" + a
    else:
        a = "0" + a
    
    if b_neg:
        b = "1" + b
    else:
        b = "0" + b
    
    #print(a)
    #print(b)
    
    idx = 0
    for i in range(1, n_bits - 1, 3):
        if a[idx] == "1":
            qc.x(i)
        idx += 1
    
    toff = []
    
    idx = 0
    for i in range(2, n_bits - 1, 3):
        if b[idx] == "1":
            qc.x(i)
            
        toff.append(i)
        
        idx += 1
    
    qc.x(n_bits - 1)
    
    for i in range(n_bits):
        qc.barrier(i)
    
    for i in range(0, n_bits - 4, 3):
        add_oracle(qc, i)
    
    toff.append(n_bits - 1)
    
    if len(toff) - 1 == 4:
        gate = C4XGate()
    elif len(toff) - 1 == 3:
        gate = C3XGate()
    
    #print(gate)
    #print(toff)
    
    for i in range(len(toff) - 1):
        qc.x(toff[i])
    
    qc.append(gate, toff)
    
    qc.measure(n_bits-1, 0)
    
    print(qc)
    
    # Execute the circuit on a simulator backend
    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, shots = 40).result()
    counts = result.get_counts(qc)
    
    
    print(counts)
    
    # Return the result
    if '0' in counts or a_num < 0:
        return b_num
    else:
        return a_num

In [201]:
find_the_largest_number(-6, 3)

        ░                                                                      »
 q_0: ──░───────■────■─────────────────────────────────────────────────────────»
      ┌───┐ ░   │    │                                                         »
 q_1: ┤ X ├─░───┼────┼────■────■───────────────────────────────────────────────»
      └─░─┘ ░ ┌─┴─┐  │  ┌─┴─┐  │  ┌───┐                                        »
 q_2: ──░─────┤ X ├──■──┤ X ├──■──┤ X ├────────────────────────────────────────»
        ░     └───┘┌─┴─┐└───┘┌─┴─┐└───┘                                        »
 q_3: ──░──────────┤ X ├─────┤ X ├──■────■─────────────────────────────────────»
      ┌───┐ ░      └───┘     └───┘  │    │                                     »
 q_4: ┤ X ├─░───────────────────────┼────┼────■────■───────────────────────────»
      └─░─┘ ░                     ┌─┴─┐  │  ┌─┴─┐  │  ┌───┐                    »
 q_5: ──░─────────────────────────┤ X ├──■──┤ X ├──■──┤ X ├────────────────────»
        ░                   

3